## This notebook is meant to teach you the basics of probability theory using some direct examples with Python. 

# Imports

In [ ]:
import numpy as np
import scipy
import scipy.stats
import pandas
import matplotlib.pyplot as plt

# Read in the data:

In [ ]:
data_filename='https://raw.githubusercontent.com/uofscphysics/STEM_Python_Course/Summer2020/02_Week2/Data/RandomVariable_Generated_Data.dat'
data = pandas.read_csv( data_filename,sep=' ',header=0)
print ( data )

# One useful helper function for plotting:

In [ ]:
def SimpleColorPlotFromFunc( 
    Func2D = None,
    xmin = None,
    xmax = None, 
    ymin = None,
    ymax = None, 
    ):

    #Make the list of poitns to plug in from the boundaries:
    x = np.linspace(xmin, xmax, 100)
    y = np.linspace(ymin, ymax, 100)
    X, Y = np.meshgrid(x, y)
    PointsToPlugIn = np.vstack([X.ravel(), Y.ravel()])
    PointsToPlugInDataset = PointsToPlugIn.T


    #plug in the list of points:
    FunctionResultValuesForGrid = []
    for Point in PointsToPlugInDataset:
        Value = Func2D(Point)
        FunctionResultValuesForGrid.append(Value)

    #reshape stuff in a confusing way so matplotlib can think of the data like a matrix
    Z = np.reshape(FunctionResultValuesForGrid, X.shape)


    #Actually construct the figure...
    plt.figure()
    heatmap = plt.imshow( 
        np.rot90(Z), 
        extent=[xmin, xmax, ymin, ymax] ,
        aspect = 'auto' ,
        interpolation = None,
        )  
    

# Marginal Distributions:

### Make a histogram of the data from column A

In [ ]:
# Here we have marginalized over the "B" parameter and are looking at the marginal "A" distribution
Adata = data['A'] # just create an array of the "A" data
plt.hist(Adata, density=True,bins=50,label='A') #plot a normalized histogram with 50 bins
plt.xlabel('A')
plt.ylabel('Number')
plt.legend()
plt.show()

### Make a histogram of the data from column B

In [ ]:
# Here we have marginalized over the "A" parameter and are looking at the marginal "B" distribution
Bdata = data['B'] # just create an array of the "B" data
plt.hist(Bdata, density=True,bins=50,label='B') # plot a normalized histogram with 50 bins
plt.xlabel('B')
plt.ylabel('Number')
plt.legend()
plt.show()

### Approximate the data from column A as a univariate gaussian: (MARGINAL)

In [ ]:
# Here we just use the measured mean and standard deviation of the Adata to approximate the
# marginal distribution as a 1D Gaussian distribution
A_standard_deviation = np.std( Adata ) 
A_mean = np.mean(Adata)

ATrialPoints = np.linspace(-10, 15, 100) # Get some trial points to test our Gaussian
                                         # approximation (linspace makes 100 points between)
                                         # -10 and 15
AValuePoints = scipy.stats.norm.pdf(ATrialPoints, loc = A_mean, scale = A_standard_deviation )

plt.hist(Adata, density=True, bins=50,label='Binned A')
plt.plot(ATrialPoints, AValuePoints,label='Gaussian A')
plt.xlabel('A')
plt.ylabel('Number')
plt.legend()
plt.show()

### Approximate the data from column B as a univariate gaussian: (MARGINAL)

In [ ]:
# Same as the previous cell, but now for the B data array
B_standard_deviation = np.std( Bdata )
B_mean = np.mean(Bdata)

BTrialPoints = np.linspace(-10, 20, 100) # Get some trial points to test our Gaussian
                                         # approximation (linspace makes 100 points between)
                                         # -10 and 20
BValuePoints = scipy.stats.norm.pdf(BTrialPoints, loc = B_mean, scale = B_standard_deviation )

plt.hist(Bdata, density=True, bins=50,label='Binned B')
plt.plot(BTrialPoints, BValuePoints,label='Gaussian B')
plt.xlabel('B')
plt.ylabel('Number')
plt.legend()
plt.show()

# Conditional distributions

### (1) Approximate the joint probability density function of A and B with a multivariate gaussian


In [ ]:
# In the previous 2 cells, we approximated each marginal distribution as a 1D Gaussian, with
# varying results. Now we can combine the A and B data into a single array and approximate
# the "joint probability density" as a 2D Gaussian.

ABdata = np.vstack( ( data['A'], data['B'] ) ).T # vstack simply stacks a list of arrays
                                                 # so we give it a list of the A and B data.
                                                 # The ".T" performs a transpose, because
                                                 # by default this will give us the data in
                                                 # 2 rows and 1000 columns, we want the opposite.
print('ABdata:', ABdata)
print('ABdata shape:', ABdata.shape)

ABMean = np.mean( ABdata, axis = 0 ) # axis = 0 says to take the mean along the "row" axis, so that
                                     # we end up with an "A" mean and a "B" mean. 
print ('ABdata Mean:',ABMean)

ABCovarianceMatrix = np.cov( ABdata, rowvar = False ) # A 2D Gaussian is defined with a covariance
                                                      # matrix instead of a standard deviation.
print ('ABdata Covariance Matrix:',ABCovarianceMatrix)

# Let's define a function that simply takes a point (A,B) and returns the result 
# of the 2D Gaussian defined by the means/covariance matrix above at that point.
def jointGaussian(ABpoint):
    return scipy.stats.multivariate_normal.pdf( ABpoint, ABMean, ABCovarianceMatrix )

print('2D Gaussian result:',jointGaussian([0,0]))

SimpleColorPlotFromFunc(jointGaussian,
                        xmin = np.min(Adata),
                        xmax = np.max(Adata),
                        ymin = np.min(Bdata),
                        ymax = np.max(Bdata))
plt.title("Probability Density of A & B", fontsize=30)
plt.ylabel('B',fontsize=40)
plt.xlabel('A',fontsize=40)
plt.draw()
plt.show()

### (1)  Fixing A = 0, plot the unnormalized conditional probability density of B:
P(B|A)
=====


In [ ]:
# The conditional probability density of "B given A=0", means setting the value of A in our 
# 2D distribution equal to 0 and observing the distribution of B values for that restriction

# We can define a function that calculates the B probability density, conditioinal
# on A=0, unnormalized. It simply calls our "jointGaussian" function above with A=0 and
# any B value. 
def BdensityConditionalOnA0_unnormalized(Bpoint):
    return jointGaussian( [0, Bpoint] )

testB_Values=np.linspace(np.min(Bdata),np.max(Bdata),1000) # get a range of B values to plot
BdensityConditionalOnA_Values=[] # this will be marginal P(B|A=0)
# Let's calculate the conditional probability given A=0 for each one of the B
# values in "testB_Values"
for B_value in testB_Values: 
    BdensityConditionalOnA_Values.append(BdensityConditionalOnA0_unnormalized(B_value))
plt.plot(testB_Values,BdensityConditionalOnA_Values,label='A=0')
plt.ylabel('P(B|A)',fontsize=30)
plt.xlabel('B',fontsize=30)
plt.legend(fontsize=15)
plt.show()

### (1) Find the conditional probability density of B for a range of A values.


In [ ]:
# Now let's do the same thing but for a range of different A values. Compare the plot at the end
# with the 2D distribution we created above to make sure you understand what we are plotting here
# and what it means. 

A_fixed_values=[1,3,5,7] 

testB_Values=np.linspace(np.min(Bdata),np.max(Bdata),1000)

# We can loop through the A_fixed_values list above,
# and essentially create the same list and plot as the 
# previous cell for each iteration. Therefore on the 
# first iteration for example, we will be finding the 
# conditional probability density of B given that A=1.
for Afixed in A_fixed_values:
    # We can create the same function used in the 
    # previous cell, except now we set the A value to
    # Afixed (which is changing on every iteration)
    # instead of simply A=0.
    def BdensityConditionalOnA_unormalized(Bpoint):
        return jointGaussian( [Afixed, Bpoint] )
    BdensityConditionalOnA_Values=[]
    for B_value in testB_Values:
        BdensityConditionalOnA_Values.append(BdensityConditionalOnA_unormalized(B_value))
    plt.plot(testB_Values,BdensityConditionalOnA_Values,label='A='+str(Afixed))

plt.legend(fontsize=15)
plt.ylabel('P(B|A)',fontsize=30)
plt.xlabel('B',fontsize=30)
plt.show()


### (1) Fixing B = 0 plot the unnormalized probability density of A:


In [ ]:
# This is now the exact same process as above (setting A=0), except now we find
# the conditional probability density for A, given that B=0.

def AdensityConditionalOnB0_unnormalized(Apoint):
    return jointGaussian( [Apoint, 0] )

A_values=np.linspace(np.min(Adata),np.max(Adata),1000)
AdensityConditionalOnB_Values=[] # this will be marginal P(B|A=0)
for A_value in A_values:
    AdensityConditionalOnB_Values.append(AdensityConditionalOnB0_unnormalized(A_value))
plt.plot(A_values,AdensityConditionalOnB_Values,label='B=0')

plt.ylabel('P(A|B)',fontsize=30)
plt.xlabel('A',fontsize=30)
plt.legend(fontsize=15)
plt.show()
